In [1]:
import numpy as np
import cvxpy as cp
from collections import defaultdict

## Bipartite matching scheduling formulation
In this notebook, we investigate a bipartite matching reformulation of the 7-day scheduling problem. Rather than assigning individual people to test schedules, we observe that there are only $2^7 - 8 = 120$ unique, valid student schedules (there are only 120 binary strings of length 7 with at least two 1s) and 21 testing schedules (there are only 21 binary strings of length 7 with exactly two 1s). These unique student schedules can be weighted according to their prevalence, and we can impose constraints to avoid overloading any particular testing day.

One disadvantage of this approach is that, at least naively, the number of nodes on each side of the bipartite graph is exponential (though the number of person schedules is bounded by the number of people); this can potentially be alleviated by breaking the semester scheduling problem up into smaller problems and proving some bounds on the optimality of the disjoint solution. Also, this approach is not dynamic according to the criteria Moon described. However, it has the following advantages:
* It scales according to the number of days, *not* according to the number of students.
* The problem size is quite small for the 7-day case.
* We can use it to find a scheduling that minimizes the **mean squared deviation from the testing interval**—that is, if we desire Q3.5 testing, then person-test assignments with a 3-day or 4-day testing interval are cheap, and person-test assignments with a 1-day or 6-day testing interval are quite expensive.

In [2]:
n_days = 7
test_interval = 3.5
n_tests = n_days / test_interval
buffer = .05

In [3]:
person_schedules = ['{:07b}'.format(i) for i in range(2 ** n_days)
                    if '{:b}'.format(i).count('1') >= n_tests]
test_schedules = ['{:07b}'.format(i) for i in range(2 ** n_days)
                    if '{:b}'.format(i).count('1') == n_tests]

In [4]:
test_costs = np.zeros(len(test_schedules))
test_days = np.zeros((n_days, len(test_schedules)))
test_schedule_intervals = []
for idx, sched in enumerate(test_schedules):
    interval = sched[sched.index('1') + 1:].index('1') + 1
    test_schedule_intervals.append(interval)
    test_days[sched.index('1'), idx] = 1
    test_days[sched.index('1') + interval, idx] = 1
    test_costs[idx] = (interval - test_interval)**2

In [5]:
test_costs

array([6.25, 2.25, 6.25, 0.25, 2.25, 6.25, 0.25, 0.25, 2.25, 6.25, 2.25,
       0.25, 0.25, 2.25, 6.25, 6.25, 2.25, 0.25, 0.25, 2.25, 6.25])

In [6]:
def compatible(person_schedule, test_schedule):
    return not any(t == '1' and p == '0'
                   for t, p in zip(test_schedule, person_schedule))

pairwise_costs = np.zeros((len(person_schedules), len(test_schedules)))
for p_idx, p_sched in enumerate(person_schedules):
    for t_idx, t_sched in enumerate(test_schedules):
        if compatible(p_sched, t_sched):
            pairwise_costs[p_idx, t_idx] = test_costs[t_idx]
        else:
            pairwise_costs[p_idx, t_idx] = 100000

In [7]:
np.random.seed(0)
demand_per_schedule = np.random.randint(0, 100, len(person_schedules))
max_supply_per_day = demand_per_schedule.sum() / test_interval

#### Variables
* $x_{ij}$ – the assignment of person schedule $i$ to testing schedule $j$ and $c_{ij}$.
* $c_{ij}$ – the unit cost of assignment $x_{ij}$. We fix $c_{ij} = \infty$ (in practice, a very large constant) if the person schedule is incompatible with the testing schedule, and we fix $c_{ij}$ to the score (the mean squared deviation from the testing interval) of the testing schedule otherwise.
* $w_i$ – the weight (number of people) of person schedule $i$. 
* $n$ – number of person schedules
* $m$ – the number of testing schedules.
* $d$ - expected supply per day
* $\alpha$ - daily supply tolerance
* $D$ - a binary matrix encoding the days used in each testing schedule (dimension $7 \times m$)


#### Problema

\begin{gather*}
\min \quad \sum_{i=1}^{n} \sum_{j=1}^{m} x_{ij} c_{ij} \\
\begin{aligned}
\text{s.t. } x_{ij} \geq 0 \qquad &\forall i \in [1, n], j \in [1, m] &\text{(non-negativity)} \\
\sum_{j=1}^m x_{ij} = w_i \qquad &\forall i \in [1, n] &\text{(demand is satisfied)} \\
\sum_{i=1}^n x_{ij} \geq (1 - \alpha)d \qquad &\forall j \in [1, m] &\text{(minimum daily supply)} \\
\sum_{i=1}^n x_{ij} \leq (1 + \alpha)d \qquad &\forall j \in [1, m] &\text{(maximum daily supply)} 
\end{aligned}
\end{gather*}

In [8]:
assignment = cp.Variable((len(person_schedules), len(test_schedules)), integer=True)
objective = cp.Minimize(cp.sum(cp.multiply(pairwise_costs, assignment)))
constraints = [
    assignment >= 0,
    cp.sum(assignment, axis=1) == demand_per_schedule,
    test_days @ cp.sum(assignment, axis=0) <= (1 + buffer) * max_supply_per_day,
    test_days @ cp.sum(assignment, axis=0) >= (1 - buffer) * max_supply_per_day,
]
prob = cp.Problem(objective, constraints=constraints)
prob.solve()

Using license file /Users/pjrule/gurobi.lic
Academic license - for non-commercial use only


4798.5

In [9]:
sol = np.array(assignment.value)
daily_demands = test_days @ sol.sum(axis=0)
print('Daily demand (min):', daily_demands.min())
print('Daily demand (max):', daily_demands.max())
print('All daily demands:', daily_demands)

Daily demand (min): 1606.0
Daily demand (max): 1774.0
All daily demands: [1667. 1606. 1774. 1774. 1627. 1774. 1606.]


In [10]:
sol_interval_counts = defaultdict(int)
for idx, count in enumerate(sol.sum(axis=0)):
    sol_interval_counts[test_schedule_intervals[idx]] += count

In [11]:
print('--- Tests by spacing ---')
for interval, count in sol_interval_counts.items():
    print('{:d} days: {:d}'.format(interval, int(count)))

--- Tests by spacing ---
1 days: 283
2 days: 537
3 days: 3094
4 days: 1832
5 days: 115
6 days: 53
